In [1]:
import os, sys
import torch
import torchvision
import torch.nn as nn

### Setting block

In [105]:
def get_num_parameters(model):
    total_num_parameters = 0
    for name, paras in model.named_parameters():
        total_num_parameters += paras.numel()
        
    return total_num_parameters

In [101]:
#Reference: https://github.com/ericsun99/Shufflenet-v2-Pytorch/blob/master/ShuffleNetV2.py
def channel_shuffle(x, groups):
    b, c, h, w = x.size() # (B, C, H, W)
    
    assert c % groups == 0, "Groups Error: number of channels should be divisible by groups"

    channels_per_group = c // groups
    
    """ reshape for channel shuffling """
    out = x.view(b, groups, channels_per_group, h, w)
    ourt = torch.transpose(out, 1, 2).contiguous()
    out = x.view(b, -1, h, w)

    return out

class Basic_conv2d_LeakyReLU(nn.Module):
    def __init__(self, in_channels, out_channels, inplace, **kwarg):
        super(Basic_conv2d_LeakyReLU, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, **kwarg)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=inplace)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.leaky_relu(out)
        
        return out

class depthwise_conv(nn.Module):
    def __init__(self, in_channels, kernels_per_layer, out_channels, groups, **kwarg):
        super(depthwise_conv, self).__init__()
        
        assert in_channels%groups == 0, "Groups Error: in_channels should be divisible by groups"
            
        self.depthwise_conv = nn.Conv2d(in_channels=in_channels,
                                        out_channels=kernels_per_layer*in_channels,
                                        groups=in_channels,
                                        **kwarg)
        
    def forward(self, x):
        out = self.depthwise_conv(x)
        
        return out

class ShuffleNet_V2_Block(nn.Module):
    def __init__(self, in_channels, out_channels, strides, groups):
        super(ShuffleNet_V2_Block, self).__init__()
        assert in_channels == out_channels, 'input channels should be the same as output channels'
        
        self.groups = groups
        self.strides = strides
        
        half_channel = out_channels // 2
        
        if strides == 1 or strides == (1,1):
            self.branch1 = nn.Sequential(
                Basic_conv2d_LeakyReLU(in_channels//2, half_channel, True, kernel_size=(1,1), stride=1, padding=0),
                depthwise_conv(half_channel, 1, half_channel, half_channel, kernel_size=(3,3), stride=strides, padding=1),
                Basic_conv2d_LeakyReLU(half_channel, half_channel, True, kernel_size=(1,1), stride=1, padding=0)
            )
            
        else:
            self.branch1 = nn.Sequential(
                depthwise_conv(in_channels, 1, in_channels, in_channels, kernel_size=(3,3), stride=strides, padding=1),
                Basic_conv2d_LeakyReLU(in_channels, half_channel, False, kernel_size=(1,1), stride=1, padding=0)
            )
            
            self.branch2 = nn.Sequential(
                Basic_conv2d_LeakyReLU(in_channels, half_channel, False, kernel_size=(1,1), stride=1, padding=0),
                depthwise_conv(half_channel, 1, half_channel, half_channel, kernel_size=(3,3), stride=strides, padding=1),
                Basic_conv2d_LeakyReLU(half_channel, half_channel, False, kernel_size=(1,1), stride=1, padding=0)
            )
            
    def forward(self, x):
        if self.strides == 1 or self.strides == (1,1):
            x1, x2 = torch.chunk(x, chunks=2, dim=1) #split into half along the channel
            out = torch.cat((x1, self.branch1(x2)), dim=1)
            
        else:
            out = torch.cat((self.branch1(x), self.branch2(x)), dim=1)
            
        return channel_shuffle(out, groups=self.groups)
            

### Testing

In [102]:
x = torch.randn(64, 4, 300, 300)
x.size()

torch.Size([64, 4, 300, 300])

In [103]:
model = ShuffleNet_V2_Block(in_channels=4, out_channels=4, strides=2, groups=2)
model

ShuffleNet_V2_Block(
  (branch1): Sequential(
    (0): depthwise_conv(
      (depthwise_conv): Conv2d(4, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=4)
    )
    (1): Basic_conv2d_LeakyReLU(
      (conv): Conv2d(4, 2, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
  )
  (branch2): Sequential(
    (0): Basic_conv2d_LeakyReLU(
      (conv): Conv2d(4, 2, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
    )
    (1): depthwise_conv(
      (depthwise_conv): Conv2d(2, 2, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2)
    )
    (2): Basic_conv2d_LeakyReLU(
      (conv): Conv2d(2, 2, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [106]:
get_num_parameters(model)

98

In [104]:
out = model(x)
out.size()

torch.Size([64, 4, 150, 150])

### Reference:
1. [ShuffleNet V2: Paper](https://arxiv.org/pdf/1807.11164.pdf)
2. [ShuffleNet V2: Pytorch Tutorial](https://github.com/ericsun99/Shufflenet-v2-Pytorch/blob/master/ShuffleNetV2.py)
3. [ShuffleNet V2: Review Article](https://zhuanlan.zhihu.com/p/48261931)
4. [ShuffleNet V1: Paper](https://arxiv.org/pdf/1707.01083.pdf)
5. [ShuffleNet V1: Review Article](https://blog.csdn.net/hongbin_xu/article/details/84304135)
6. [SqueezeNet MobileNet ShuffleNet Xception: Review Article](https://zhuanlan.zhihu.com/p/32746221)